In [2]:
from llama_index.core.query_engine import MultiStepQueryEngine
from LLM import LLM
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

c:\USERS\AAI47\CHATBOT_POC1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import os
import torch
from transformers import pipeline

from llama_index.schema import IndexNode
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.llms import OpenAI
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.response_synthesizers import get_response_synthesizer
from llama_index.vector_stores import PineconeVectorStore
from llama_index.graph_stores import Neo4jGraphStore

ModuleNotFoundError: No module named 'llama_index.schema'

In [112]:
from llama_index.core import Settings
from CustomEmbedding import CustomAPIEmbeddings
Settings.llm=LLM()
embed_model = CustomAPIEmbeddings(embed_batch_size=2)
Settings.embed_model = embed_model



Define utterances

In [113]:
ds_doc_queries=[
        "What are the steps and options available to administrators for customizing the search and tagging functionalities within the 3DEXPERIENCE platform?",
        "How do I manage and refine searches within the 3DEXPERIENCE platform?",
        "How do I manage and refine searches within the 3DEXPERIENCE platform?",
        "How to select and customize view modes in 3DSearcht",
        "How to handle search settings and errors in 3DSearch:",
        "How to utilize search result actions like tagging, downloading, and editing in 3DSearch",
        "How to sort and export search results in 3DSearch",
        "How to manage column customizations in Datagrid view in 3DSearch",
        "How to select multiple objects in 3DSearch",
        "What is the purpose of the topbar in the 3DEXPERIENCE platform",
        "How to refine search queries using advanced search options in the topbar",
        " How to select and customize view modes in 3DSearch",
        "can i use Advanced Search",
        "How do I refine searches within the 3DEXPERIENCE platform, including performing and refining searches, selecting and customizing view modes, and utilizing search result actions?",
        "What are the key features of the 6WTags functionality, including filtering search results, managing tags in widgets, and utilizing app-specific integrations?",
        "What are the steps and options available to administrators for customizing the search and tagging functionalities within the 3DEXPERIENCE platform?",
        "How do I manage and filter content using six categories: What, Who, When, Where, How, and Why?",
    ]

federated_search_queries=[
       "Show all documents created by John Doe in January 2024.",
        "List all 3D shapes modified before July 10, 2024, by any engineer.",
        "Give me all social content titled 'Team Building Exercise' created by Idir",
        "Retrieve all documents with a .docx extension edited by Jane.",
        "Find posts from 2024 that have received more than 10 likes.",
        "Display all personnel in the Data Scientist role who have created documents this year.",
        "Show me every vpmreference product updated in the last month.",
        "Which documents related to 'AI Project' were last modified by a Vice President?",
        "List all the ideas that have no comments as of today.",
        "Find all profiles of engineers who modified any social content since June 2024.",
        "What are the latest documents created by Idir",
        "Show all products that have been labeled as prototypes and modified this week.",
        "Retrieve details on social posts made by Alice that include 'market trends'.",
        "retrieve 3D model created on February 1, 2024",
        "List all custom documents created last year.",
        "Find all users who posted ideas related to 'sustainability' in 2024.",
        "Show me documents with more than 5 revisions made in the first quarter of 2024.",
        "Which VpmReferences are classified under the root content structure?",
        "Retrieve all social content created by the R&D department.",
        "Display the document titled 'Safety Protocols'.",
        "List all physical products created by any member of the Strategy & Management team.",
        "Who modified the most documents in the month of May 2024?",
        "Find all posts with exactly 2 endorsements that discuss 'new technology'.",
        "Show all business roles related to the 'Engineer' category updated last week.",
        "Retrieve all documents linked to 'Product Design' that were last modified yesterday.",
        "List everyone who created a PLMDocCustom document in March 2024.",
        "Which documents about 'Project Alpha' were last reviewed by software engineers?",
        "Find all social posts from 2024 discussing 'team collaboration'.",
        "Display all physical products classified under Intermediate content structure created in 2024.",
        "List all VPMReferences created by people in the 'Data Scientist' role.",
        "Retrieve the latest modified VPMReference by any Vice President.",
        "Show me all personnel changes within the Engineering department this year.",
        "Which social content received the highest number of likes last month?",
        "Find documents with 'confidential' in their descriptions updated by strategy managers.",
        "Display all posts made by any person named 'Tom' that have at least 3 comments.",
        "List all engineering documents that include the term 'prototype' in their title.",
        "show documents related to 'Regulatory Compliance' in 2024?",
        "Find all document with extension 'doc' or 'pdf'.",
        "Display the documents created by the Software Engineer in 2024.",
]
simulation_queries=[
        "Simulate airflow around the new car design model.",
        "Run a stress test on the bridge structure model from the latest dataset.",
        "Initiate a thermal simulation for the VPMReference of the jet engine.",
        "Show me the simulation results of the earthquake resistance test for building model B23.",
        r"Apply a 10% increase in load to the wing structure and rerun the simulation.",
        "Create a 3D simulation showing the wear and tear over 5 years for the gear assembly.",
        "Display the vibration analysis for the prototype machine model under operational conditions.",
        "Simulate water flow through the new pipe design and identify potential pressure drop points.",
        "Run a dynamic simulation for the robotic arm movements when lifting a 100 kg weight.",
        "Perform a crash test simulation for the latest car chassis design.",
        "Generate a color stress map on the VPMReference of the suspension bridge under maximum load.",
        "Show a real-time simulation of the assembly line for product X using its 3D model.",
        "Execute a fatigue analysis on the turbine blades model based on current operational data.",
        "Simulate the dispersion pattern of particles in the air filter system.",
        "Preview the heat dissipation simulation for the newly designed laptop cooling system.",
        "Conduct a light exposure simulation on the solar panel array model.",
        "Perform a simulation to test the new packaging design's resistance to shock and drops.",
        "Create a simulation of the airflow in the HVAC system for the commercial building model.",
        "Run a fluid dynamics simulation for the prototype water pump.",
        "Simulate the electrical conductivity of the new semiconductor design.",
        "Preview the deformation under load for the composite material sample.",
        "Initiate a magnetic field simulation for the electric motor model.",
        "Perform a wind tunnel simulation on the scale model of the high-speed train.",
        "Display the simulation of sound wave propagation through the new theater design.",
        "Simulate the effect of different soil types on the foundation model of the skyscraper.",
        "Run a multi-phase flow simulation for the oil pipeline model under varying pressures.",
        "Show the thermal expansion simulation for the metal alloy under extreme heat.",
        "Perform a particle collision simulation in the new accelerator model.",
        "Create a 3D simulation of light absorption by the new photovoltaic material.",
        "Simulate the mechanical behavior of the joint assembly under repetitive movements.",
        "Execute a corrosion simulation for the underwater hull design of the ship.",
        "Run an aerodynamics simulation for the new drone design at different wind speeds.",
        "Display a simulation of the wireless signal coverage for the new smartphone design.",
        "Simulate the operational efficiency of the hydraulic system under full load.",
        "Perform a shadow analysis on the urban development model during summer solstice.",
        "Initiate a resonance frequency simulation for the newly designed guitar.",
        "Show the simulation of fluid interaction with different impeller designs in the mixer.",
        "Run a granular flow simulation for the hopper design in the food processing plant.",
        "Simulate the dynamic response of the suspension system when driving over rough terrain.",
        "Perform a virtual crash test for the latest design.",
]



In [134]:
query="How to use the search tool to filter ? and then search for me all posts created today"
# query="Launch simulation about content design created yesterday"
# query="what is ds6w"
# query="search for content design created yesterday then launch a simulation about these results"

prompt = f"""
You are an intelligent agent designed to decompose user queries to successfully route them to the most relevant agent based on the task and intent of the query. 

We have three agents:
1. The Informative Agent retrieves information about how to use the search and filters, definitions, checks for accessibility and access, and responds to user queries from web documentation.
2. The Search Agent launches searches for desired items and can filter by date, person, and other criteria.
3. The Act Agent performs simulations.

Here are the steps to handle the query:
1. Analyze the given user query to determine its intent, considering the global context:
   - If the query has a single intent, reformulate it clearly without changing its core meaning. Correct or clarify it as needed, but do not make significant modifications.
   - If there are multiple intents or hidden intents, decompose the query into sub-queries where each sub-query has a single intent and single task.
2. Determine the final and global intent of the result.
3. If decomposition is necessary, decompose the query into sub-queries without adding extra information or changing the original meaning. Ensure the intent/task is clearly expressed and order these tasks by time of execution: define which task should be launched first and which comes after it.

Important: For single intent queries, do not significantly alter the original query. Only make necessary corrections or clarifications. Do not add extra information or change the intent.

Return the result in the following format:
- If single intent: {{"1": "REWRITTEN_QUERY"}}
- If multiple intents: {{"1": "FIRST_SUBQUERY/FIRST_TASK", "2": "SECOND_SUBQUERY/SECOND_TASK", ...}} with the keys indicating the order of execution.

Examples:

Original query: "launch a simulation about content design created today"
This is a multi-intent query. 
The global intent is to perform a simulation, but a simulation of what? 
That is why we need to search for the object to simulate, which involves searching for content designs created today.

Result:
{{
  "1": "Search for content designs created today.",
  "2": "Execute a simulation of these results."
}}

Another Example: 
Original query: "How to launch advanced search"
This is a single-intent query. 
The global intent is to look for information.

Result:
{{"1": "How to launch advanced search?"}}

Respond ONLY with the result. Do not Add extra text

Original query: {query}
"""

llm = LLM()
processed_query = llm.complete(prompt).text
processed_query


'{\n  "1": "How to use the search tool to filter?",\n  "2": "Search for all posts created today."\n}'

### Query Routing using LLM Multiselector

In [111]:
from llama_index.core.tools import ToolMetadata
from llama_index.core.selectors import LLMSingleSelector,LLMMultiSelector, PydanticMultiSelector


choices = [
    ToolMetadata(description="Documentation, informative assitant on What and how to use the 3dSearch and 6wtags withing to 3dExperience platform for simple users, developer and admins ", name="poc1"),
    ToolMetadata(description="useful for searching and filtering using natural language for example users can search for objects and filter by person responsible, creation date, modification date , type of object ,content,description and title   ", name="poc2"),
    ToolMetadata(description="An Act assitant , that do simulations of object, receive in input and object for example a products, a description , and convert it to 3d design ou do a simulation of it ", name="poc3"),
]
selector =  LLMMultiSelector.from_defaults()
selector_result = selector.select(
    choices, query="launch simulation about content created today"
)
print(selector_result.selections)

[SingleSelection(index=2, reason='The Act assistant can simulate objects, including receiving input and converting it to 3D design, which could be relevant to launching a simulation about content created today.')]


In [145]:
import os

doc = r"intention _ngine\queries\doc_queries.txt"
search = r"intention_engine\queries\Federated_search_queries.txt"
simulation = r"intention_engine\queries\simulation_queries.txt"

print(os.listdir(doc))
print(os.listdir(search))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'intention engine\\queries\\doc_queries.txt'

In [ ]:
admin_engine

In [136]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=admin_engine,
        metadata=ToolMetadata(
            name="Admin ",
            description=(
                "Steps and options available to administrators for customizing the search and tagging functionalities within the 3DEXPERIENCE platform, including synonym management, tag inclusion and exclusion, default display settings, and the ability to import and export search configurations for platform-wide user experience optimization "
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=user_6w_engine,
        metadata=ToolMetadata(
            name="User 6WTags documentation",
            description=(
                "The 3DEXPERIENCE platform documentation on 6WTags provides comprehensive information on managing and filtering content using six categories: What, Who, When, Where, How, and Why. These tags can be applied to various types of content, including design data, simulation results, and office documents. The documentation explains how to use and customize 6WTags, create and remove user-defined tags, and integrate 6WTags with specific applications like 3DDashboard, 3DSearch, and 3DSwym. Key features include filtering search results, managing tags in widgets, and utilizing app-specific integrations to enhance content organization and retrieval. This functionality allows users to efficiently categorize and find relevant data within the platform."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=user_3dsearch_engine,
        metadata=ToolMetadata(
            name="User 3DSearch documentation",
            description=(
                "The User 3DSearch documentation provides comprehensive guidance on managing and refining searches within the 3DEXPERIENCE platform. It covers performing and refining searches, selecting and customizing view modes, sorting and exporting results, and utilizing search result actions like tagging, downloading, and editing. Key features include the 6W Lifelike Search Experience, Advanced Search for filtering with specific attributes, focusing on preferred types, and using predefined queries for targeted searches. Additionally, it explains selecting multiple objects, managing column customizations in Datagrid view, and handling search settings and errors. Users can call back the last search, add favorite searches, and access personal indexed content with My Content and My Recent Content features. These functionalities collectively enhance the efficiency and accuracy of data retrieval, ensuring effective search management and retrieval within the platform."
            ),
        ),
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools
)


TypeError: __init__() missing 1 required positional argument: 'query_engine'